<a href="https://colab.research.google.com/github/aadi0004/Deep-Learning/blob/main/Vector_db(chromadb).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb google-generativeai sentence-transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 10.9 MB/s e

In [ ]:
# Set up our gemini api key

In [ ]:
import google.generativeai as genai
import os
os.environ["Google_API_KEY"] = " "
genai.configure(api_key = os.getenv("Google_API_KEY"))

In [ ]:
# Initialize Cromadb

# import chromadb

# # Initialize ChromaDB client (in-memory or persistent storage)
# chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Persistent storage
# # chroma_client = chromadb.Client()  # In-memory storage

# # Create a new collection
# collection = chroma_client.get_or_create_collection(name="gemini_embeddings")


In [ ]:
import chromadb
chroma_client = chromadb.PersistentClient(path = "./chroma_db")
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="gemini_embeddings")

In [ ]:
# 4. Generate Text Embeddings using Sentence Transformers

# Since Google Gemini does not provide direct embeddings, we use sentence-transformers to generate embeddings from text.

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def generate_embedding(text):
  return embedding_model.encode(text).tolist()

In [ ]:
# 5. Insert Data into ChromaDB

In [ ]:
documents = [
    {"id": "1", "text": "What is machine learning?"},
    {"id": "2", "text": "Explain deep learning and its applications."},
    {"id": "3", "text": "What are transformers in NLP?"}
]

# Store documents with their embeddings
for doc in documents:
    embedding = generate_embedding(doc["text"])
    collection.add(
        ids=[doc["id"]],
        embeddings=[embedding],
        metadatas=[{"text": doc["text"]}]
    )

print("Documents inserted successfully!")


Documents inserted successfully!


In [ ]:
# 6. Query the Database (Semantic Search)

# To retrieve similar documents, generate an embedding for the query and perform a nearest neighbor search.

In [ ]:
# query_text = "Tell me about neural networks"
# query_embedding = generate_embedding(query_text)

# # Perform similarity search
# results = collection.query(
#     query_embeddings=[query_embedding],
#     n_results=2  # Number of closest matches
# )

# # Display search results
# for i, result in enumerate(results["metadatas"][0]):
#     print(f"Result {i+1}: {result['text']}")


In [ ]:
query_text = "Tell me What is ml"
query_embedding = generate_embedding(query_text)

## Perform similar elements
results = collection.query(
    query_embeddings = [query_embedding],

    n_results =2
)

for i , result in enumerate(results["metadatas"][0]):
  print(f"Result {i+1}: {result['text']}")

Result 1: What is machine learning?
Result 2: Explain deep learning and its applications.


In [ ]:
# 7. Use Google Gemini for Text Generation

# You can use Google Gemini API to generate answers based on the retrieved documents.

In [ ]:
def get_gemini_response(prompt):
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text

# Generate response based on retrieved data
if results["metadatas"][0]:
    context = results["metadatas"][0][0]["text"]
    response = get_gemini_response(f"Explain in detail: {context}")
    print("\nAI Response:\n", response)
else:
    print("No relevant results found.")



AI Response:
 ## Machine Learning: A Detailed Explanation

Machine learning (ML) is a subfield of Artificial Intelligence (AI) that focuses on enabling computer systems to learn from data without being explicitly programmed.  Instead of relying on hardcoded rules, machine learning algorithms identify patterns, make predictions, and improve their performance over time as they are exposed to more data.

**In essence, machine learning is about giving computers the ability to learn and adapt.**

Here's a breakdown of key aspects:

**1.  The Core Idea: Learning from Data**

*   **Data-Driven Approach:**  ML algorithms derive knowledge and insights directly from data. This data can take many forms:
    *   **Structured Data:**  Organized data stored in tables or databases with defined rows and columns (e.g., customer information, sales records).
    *   **Unstructured Data:**  Data that doesn't have a predefined format (e.g., text, images, audio, video).
*   **Pattern Recognition:**  The pr